In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 519 ms (started: 2021-08-26 20:52:58 +08:00)


In [2]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #9  Aug 26, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/keras_developer_guides-.git (fetch)
origin	git@github.com:ustchope/keras_developer_guides-.git (push)
[main 9fad412] 更新 #9  Aug 26, 2021
 2 files changed, 1971 insertions(+), 2 deletions(-)
 create mode 100644 Keras FAQ.ipynb


To git@github.com:ustchope/keras_developer_guides-.git
   98ea9e4..9fad412  main -> main


time: 4.47 s (started: 2021-08-26 20:53:17 +08:00)


In [3]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 3.01 s (started: 2021-08-26 20:53:38 +08:00)


In [4]:
tf.__version__

'2.6.0'

time: 7.15 ms (started: 2021-08-26 20:53:46 +08:00)


# 一般的问题

## 如何在多个 GPU 上（在一台机器上）训练 Keras 模型？
有两种方法可以在多个 GPU 上运行单个模型：数据并行和设备并行。 在大多数情况下，您最需要的是数据并行性。

### 数据并行

数据并行包括在每个设备上复制目标模型一次，并使用每个副本来处理输入数据的不同部分。

使用 Keras 模型进行数据并行的最佳方法是使用 tf.distribute API。 请务必阅读我们关于在 Keras 中使用 [tf.distribute](https://www.tensorflow.org/api_docs/python/tf/distribute) 的指南。

它的要点如下：

a) 实例化一个“分发策略”对象，例如 MirroredStrategy（在每个可用设备上复制您的模型并保持每个模型的状态同步）：

In [5]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
time: 1.41 s (started: 2021-08-26 21:10:46 +08:00)


b) 创建您的模型并在策略范围内编译它：

In [6]:
with strategy.scope():
    # This could be any kind of model -- Functional, subclass...
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.GlobalMaxPooling2D(),
        tf.keras.layers.Dense(10)
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

请注意，所有状态变量的创建都应在作用域下进行，这一点很重要。 因此，如果您创建任何其他变量，请在范围内进行。

c) 使用 tf.data.Dataset 对象作为输入调用 fit()。 分发与所有回调广泛兼容，包括自定义回调。 请注意，此调用不需要在策略范围内，因为它不会创建新变量。

In [ ]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

### 模型并行

模型并行包括在不同设备上运行同一模型的不同部分。 它最适用于具有并行架构的模型，例如 一个有两个分支的模型。

这可以通过使用 TensorFlow 设备范围来实现。 这是一个快速示例：

In [ ]:
# Model where a shared LSTM is used to encode two different sequences in parallel
input_a = keras.Input(shape=(140, 256))
input_b = keras.Input(shape=(140, 256))

shared_lstm = keras.layers.LSTM(64)

# 在一个 GPU 上处理第一个序列
with tf.device_scope('/gpu:0'):
    encoded_a = shared_lstm(input_a)
# 在另一个 GPU 上处理下一个序列
with tf.device_scope('/gpu:1'):
    encoded_b = shared_lstm(input_b)

# 在 CPU 上连接结果
with tf.device_scope('/cpu:0'):
    merged_vector = keras.layers.concatenate(
        [encoded_a, encoded_b], axis=-1)

## 如何在多台机器上分配训练？
TensorFlow 2 使您能够编写与分发方式基本无关的代码：任何可以在本地运行的代码都可以分发给多个工作程序和加速器，只需向其中添加与您的分发策略相对应的分发策略 (tf.distribute.Strategy)选择的硬件，无需任何其他代码更改。

这也适用于任何 Keras 模型：只需添加一个包含模型构建和编译代码的 tf.distribute 分发策略范围，训练将根据 tf.distribute 分发策略进行分发。

对于跨多台机器的分布式训练（与仅在一台机器上利用多个设备的训练相反），您可以使用两种分配策略：`MultiWorkerMirroredStrategy` 和 `ParameterServerStrategy`：
* `tf.distribute.MultiWorkerMirroredStrategy` 实现了一个同步 CPU/GPU 多线程解决方案，以使用 Keras 风格的模型构建和训练循环，使用跨副本的梯度同步减少。
* `tf.distribute.experimental.ParameterServerStrategy` 实现了一个异步的 CPU/GPU 多工作器解决方案，其中参数存储在参数服务器上，工作器异步更新梯度到参数服务器。

分布式训练比单机多设备训练涉及更多。使用 ParameterServerStrategy，您将需要启动一个由“worker”和“ps”组成的远程机器集群，每个机器都运行一个 tf.distribute.Server，然后在拥有 TF_CONFIG 环境变量的“首席”机器上运行您的 python 程序如何与集群中的其他机器通信。使用 MultiWorkerMirroredStrategy，您将在每个主管和工作人员上运行相同的程序，同样使用 TF_CONFIG 环境变量指定如何与集群通信。从那里开始，工作流程类似于使用单机训练，主要区别在于您将使用 ParameterServerStrategy 或 MultiWorkerMirroredStrategy 作为您的分发策略。

重要的是，您应该：
* 确保您的数据集配置为集群中的所有工作人员都能够有效地从中提取数据（例如，如果您的集群在 GCP 上运行，则最好将您的数据托管在 Google Cloud Storage 上）。
* 确保您的训练是容错的（例如，通过配置 keras.callbacks.BackupAndRestore 回调）。

下面，我们提供了几个涵盖基本工作流程的代码片段。 有关 CPU/GPU 多工训练的更多信息，请参阅多 GPU 和分布式训练； 对于 TPU 训练，请参阅如何在 TPU 上训练 Keras 模型？。

使用参数服务器策略：

In [ ]:
cluster_resolver = ...
if cluster_resolver.task_type in ("worker", "ps"):
  # Start a [`tf.distribute.Server`](https://www.tensorflow.org/api_docs/python/tf/distribute/Server) and wait.
  ...
elif cluster_resolver.task_type == "evaluator":
  # Run an (optional) side-car evaluation
  ...

# Otherwise, this is the coordinator that controls the training w/ the strategy.
strategy = tf.distribute.experimental.ParameterServerStrategy(
    cluster_resolver=...)
train_dataset = ...

with strategy.scope():
  model = tf.keras.Sequential([
      layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(64, activation='relu'),
      layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss='sparse_categorical_crossentropy',
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
      metrics=['accuracy'],
      steps_per_execution=10)

model.fit(x=train_dataset, epochs=3, steps_per_epoch=100)

使用 MultiWorkerMirroredStrategy：

In [ ]:
# By default `MultiWorkerMirroredStrategy` uses cluster information
# from `TF_CONFIG`, and "AUTO" collective op communication.
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
train_dataset = get_training_dataset()
with strategy.scope():
  # Define and compile the model in the scope of the strategy. Doing so
  # ensures the variables created are distributed and initialized properly
  # according to the strategy.
  model = tf.keras.Sequential([
      layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(64, activation='relu'),
      layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss='sparse_categorical_crossentropy',
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
      metrics=['accuracy'])
model.fit(x=train_dataset, epochs=3, steps_per_epoch=100)

## 如何在 TPU 上训练 Keras 模型？
TPU 是一种快速高效的深度学习硬件加速器，可在 Google Cloud 上公开使用。 您可以通过 Colab、AI 平台（机器学习引擎）和深度学习虚拟机使用 TPU（前提是在虚拟机上设置了 TPU_NAME 环境变量）。

请务必先阅读 TPU 使用指南。 这是一个快速总结：

连接到 TPU 运行时（例如，通过在 Colab 中选择 TPU 运行时）后，您需要使用 TPUClusterResolver 检测您的 TPU，它会在所有支持的平台上自动检测链接的 TPU：

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
print('Running on TPU: ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
print('Replicas: ', strategy.num_replicas_in_sync)

with strategy.scope():
    # Create your model here.
    ...

初始设置后，工作流程类似于使用单机多 GPU 训练，主要区别在于您将使用 TPUStrategy 作为您的分发策略。

重要的是，您应该：
* 确保您的数据集产生具有固定静态形状的批次。 TPU 图只能处理具有恒定形状的输入。
* 确保您能够以足够快的速度读取数据，以保持 TPU 的利用率。 使用 TFRecord 格式存储数据可能是个好主意。
* 考虑在每次图执行时运行多个梯度下降步骤，以保持 TPU 的利用率。 您可以通过experimental_steps_per_execution 参数compile() 执行此操作。 它将为小型模型带来显着的加速。

## Keras 配置文件存储在哪里？
存储所有 Keras 数据的默认目录是：

`$HOME/.keras/`

例如，对我来说，在 MacBook Pro 上，它是 `/Users/fchollet/.keras/`。

请注意，Windows 用户应将 `$HOME` 替换为 `%USERPROFILE%`。

如果 Keras 无法创建上述目录（例如由于权限问题），则使用 `/tmp/.keras/` 作为备份。

Keras 配置文件是一个 JSON 文件，存储在 `$HOME/.keras/keras.json` 中。 默认配置文件如下所示：

In [ ]:
{
    "image_data_format": "channels_last",
    "epsilon": 1e-07,
    "floatx": "float32",
    "backend": "tensorflow"
}

它包含以下字段：
* 图像处理层和实用程序（channels_last 或 channels_first）默认使用的图像数据格式。
* 用于防止在某些操作中被零除的 epsilon 数值模糊因子。
* 默认浮点数据类型。
* 默认后端。 这是遗产； 现在只有 TensorFlow。

同样，缓存数据集文件，例如使用 `get_file()` 下载的文件，默认存储在 `$HOME/.keras/datasets/` 中，来自 Keras 应用程序的缓存模型权重文件默认存储在 `$HOME/.keras/models/` .

## 如何使用 Keras 进行超参数调整？
我们推荐使用 Keras Tuner。

## 如何在开发过程中使用 Keras 获得可重现的结果？
在模型的开发过程中，有时能够从一次运行中获得可重复的结果以确定性能变化是由于实际模型或数据修改引起的，还是仅仅是新随机种子的结果是很有用的。

首先，您需要在程序启动之前（不在程序本身内）将 PYTHONHASHSEED 环境变量设置为 0。 在 Python 3.2.3 之后，这对于某些基于哈希的操作具有可重现的行为是必要的（例如，集合或字典中的项目顺序，请参阅 Python 的文档或问题 #2280 以获取更多详细信息）。 设置环境变量的一种方法是像这样启动 python 时：

In [ ]:
$ cat test_hash.py
print(hash("keras"))
$ python3 test_hash.py                  # non-reproducible hash (Python 3.2.3+)
8127205062320133199
$ python3 test_hash.py                  # non-reproducible hash (Python 3.2.3+)
3204480642156461591
$ PYTHONHASHSEED=0 python3 test_hash.py # reproducible hash
4883664951434749476
$ PYTHONHASHSEED=0 python3 test_hash.py # reproducible hash
4883664951434749476

此外，在 GPU 上运行时，某些操作具有不确定性输出，尤其是 tf.reduce_sum()。 这是因为 GPU 并行运行许多操作，因此无法始终保证执行顺序。 由于浮点数的精度有限，即使将几个数字相加也可能会产生略有不同的结果，具体取决于您添加它们的顺序。 您可以尝试避免非确定性操作，但有些操作可能由 TensorFlow 自动创建来计算梯度，因此在 CPU 上运行代码要简单得多。 为此，您可以将 CUDA_VISIBLE_DEVICES 环境变量设置为空字符串，例如：

In [ ]:
$ CUDA_VISIBLE_DEVICES="" PYTHONHASHSEED=0 python your_program.py

以下代码片段提供了如何获得可重现结果的示例：

In [ ]:
import numpy as np
import tensorflow as tf
import random as python_random

# 下面是在明确定义的初始状态下启动 Numpy 生成的随机数所必需的。
np.random.seed(123)

# 下面是启动核心 Python 生成的处于明确定义状态的随机数所必需的。
python_random.seed(123)

# 下面的 set_seed() 将使 TensorFlow 后端的随机数生成具有明确定义的初始状态。
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(1234)

# Rest of code follows ...

请注意，如果您执行上述步骤，则不必在代码中为各个初始值设定项设置种子，因为它们的种子是由上面设置的种子组合决定的。

## 我有哪些保存模型的选项？
注意：不建议使用 pickle 或 cPickle 来保存 Keras 模型。

### 全模型保存（配置+权重）

整个模型保存意味着创建一个文件，其中将包含：
* 模型的架构，允许您重新创建模型
* 模型的权重
* 训练配置（损失，优化器）
* 优化器的状态，允许您从中断的地方继续训练。

默认和推荐使用的格式是 TensorFlow SavedModel 格式。 在 TensorFlow 2.0 及更高版本中，您只需执行以下操作：`model.save(your_file_path)`。

为了明确起见，您还可以使用 `model.save(your_file_path, save_format='tf')`。

Keras 仍然支持其原始的基于 HDF5 的保存格式。 要将模型保存为 HDF5 格式，请使用 `model.save(your_file_path, save_format='h5')`。 请注意，如果 your_file_path 以 .h5 或 .keras 结尾，则会自动使用此选项。 另请参阅如何安装 HDF5 或 h5py 来保存我的模型？ 有关如何安装 h5py 的说明。

以任一格式保存模型后，您可以通过 `model = keras.models.load_model(your_file_path)` 重新实例化它。

例子：

In [8]:
from tensorflow.keras.models import load_model

model.save('my_model')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model')

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Assets written to: my_model/assets
time: 792 ms (started: 2021-08-26 21:26:17 +08:00)


### 仅保存权重

如果您需要保存模型的权重，您可以在 HDF5 中使用以下代码执行此操作：

In [ ]:
model.save_weights('my_model_weights.h5')

假设您有用于实例化模型的代码，然后您可以将保存的权重加载到具有相同架构的模型中：

In [ ]:
model.load_weights('my_model_weights.h5')

如果您需要将权重加载到不同的架构中（某些层是相同的），例如用于微调或迁移学习，您可以按层名称加载它们：

In [ ]:
model.load_weights('my_model_weights.h5', by_name=True)

例子：

In [ ]:
"""
Assuming the original model looks like this:

model = Sequential()
model.add(Dense(2, input_dim=3, name='dense_1'))
model.add(Dense(3, name='dense_2'))
...
model.save_weights(fname)
"""

# new model
model = Sequential()
model.add(Dense(2, input_dim=3, name='dense_1'))  # will be loaded
model.add(Dense(10, name='new_dense'))  # will not be loaded

# load weights from the first model; will only affect the first layer, dense_1.
model.load_weights(fname, by_name=True)

另请参阅如何安装 HDF5 或 h5py 来保存我的模型？ 有关如何安装 h5py 的说明。

### 仅配置保存（序列化）

如果您只需要保存模型的架构，而不是其权重或训练配置，您可以执行以下操作：

In [ ]:
# save as JSON
json_string = model.to_json()

生成的 JSON 文件是人类可读的，如果需要，可以手动编辑。

然后，您可以根据这些数据构建一个新模型：

In [ ]:
# model reconstruction from JSON:
from tensorflow.keras.models import model_from_json
model = model_from_json(json_string)

### 在保存的模型中处理自定义层（或其他自定义对象）

如果您要加载的模型包含自定义层或其他自定义类或函数，您可以通过 custom_objects 参数将它们传递给加载机制：

In [ ]:
from tensorflow.keras.models import load_model
# Assuming your model includes instance of an "AttentionLayer" class
model = load_model('my_model.h5', custom_objects={'AttentionLayer': AttentionLayer})

或者，您可以使用自定义对象范围：

In [ ]:
from tensorflow.keras.utils import CustomObjectScope

with CustomObjectScope({'AttentionLayer': AttentionLayer}):
    model = load_model('my_model.h5')

自定义对象处理与 load_model 和 model_from_json 的工作方式相同：

In [ ]:
from tensorflow.keras.models import model_from_json
model = model_from_json(json_string, custom_objects={'AttentionLayer': AttentionLayer})

### 我应该如何引用 Keras？
如果对您的研究有帮助，请在您的出版物中引用 Keras。 这是一个示例 BibTeX 条目：

In [ ]:
@misc{chollet2015keras,
  title={Keras},
  author={Chollet, Fran\c{c}ois and others},
  year={2015},
  howpublished={\url{https://keras.io}},
}

# 训练相关问题
## "sample", "batch", 和 "epoch"是什么意思？

以下是正确使用 Keras fit() 需要了解和理解的一些常见定义：
* 样本：数据集的一个元素。例如，一张图像是卷积网络中的一个样本。一个音频片段是语音识别模型的示例。
* 批次：一组 N 个样本。批次中的样品独立、并行处理。如果是训练，批处理只会导致模型更新一次。批处理通常比单个输入更接近输入数据的分布。批量越大，近似越好；但是，批处理将需要更长的时间来处理并且仍然只会导致一次更新，这也是事实。对于推理（评估/预测），建议在不耗尽内存的情况下选择尽可能大的批次大小（因为更大的批次通常会导致更快的评估/预测）。
* Epoch：任意截止，通常定义为“一次遍历整个数据集”，用于将训练分成不同的阶段，这对于记录和定期评估很有用。当将 validation_data 或 validation_split 与 Keras 模型的 fit 方法一起使用时，评估将在每个 epoch 结束时运行。在 Keras 中，可以添加专门设计为在 epoch 结束时运行的回调。这些示例包括学习率变化和模型检查点（保存）。

## 为什么我的训练损失比我的测试损失高得多？
Keras 模型有两种模式：训练和测试。 在测试时关闭正则化机制，例如 Dropout 和 L1/L2 权重正则化。 它们反映在训练时间损失中，而不是测试时间损失中。

此外，Keras 显示的训练损失是当前时期内每批训练数据的平均损失。 由于您的模型随着时间的推移而发生变化，因此一个 epoch 的第一批的损失通常高于最后一批的损失。 这可以降低时代平均水平。 另一方面，一个 epoch 的测试损失是使用模型计算的，因为它在 epoch 结束时，导致较低的损失。

## 如何将 Keras 与不适合内存的数据集一起使用？
你应该使用 `tf.data API` 来创建 `tf.data.Dataset` 对象——一个对数据管道的抽象，它可以从本地磁盘、分布式文件系统、GCS 等中提取数据，并有效地应用各种 数据转换。

例如，实用程序 [tf.keras.preprocessing.image_dataset_from_directory](/api/preprocessing/image#imagedatasetfromdirectory-function) 将创建一个从本地目录读取图像数据的数据集。 同样，实用程序 [tf.keras.preprocessing.text_dataset_from_directory](/api/preprocessing/text#textdatasetfromdirectory-function) 将创建一个从本地目录读取文本文件的数据集。

数据集对象可以直接传递给 fit()，也可以在自定义的低级训练循环中迭代。

In [ ]:
model.fit(dataset, epochs=10, validation_data=val_dataset)

## 我如何确保我的训练可以从程序中断中恢复？
为确保能够随时从中断的训练运行中恢复（容错），您应该使用 tf.keras.callbacks.experimental.BackupAndRestore 定期将您的训练进度（包括时期数和权重）保存到磁盘，以及 下次调用 Model.fit() 时加载它。

In [ ]:
import tensorflow as tf
from tensorflow import keras

class InterruptingCallback(keras.callbacks.Callback):
    """A callback to intentionally introduce interruption to training."""
    def on_epoch_end(self, epoch, log=None):
        if epoch == 15:
            raise RuntimeError('Interruption')

model = keras.Sequential([keras.layers.Dense(10)])
optimizer = keras.optimizers.SGD()
model.compile(optimizer, loss="mse")

x = tf.random.uniform((24, 10))
y = tf.random.uniform((24,))
dataset = tf.data.Dataset.from_tensor_slices((x, y)).repeat().batch(2)

backup_callback = keras.callbacks.experimental.BackupAndRestore(
    backup_dir='/tmp/backup')
try:
    model.fit(dataset, epochs=20, steps_per_epoch=5, 
            callbacks=[backup_callback, InterruptingCallback()])
except RuntimeError:
    print('***Handling interruption***')
    # This continues at the epoch where it left off.
    model.fit(dataset, epochs=20, steps_per_epoch=5, 
            callbacks=[backup_callback])

在回调文档中了解更多信息。

## 当验证损失不再减少时，如何中断训练？
您可以使用 EarlyStopping 回调：

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(x, y, validation_split=0.2, callbacks=[early_stopping])

在回调文档中了解更多信息。

## 如何冻结图层并进行微调？
### 设置可训练属性

所有层和模型都有一个 layer.trainable 布尔属性：

In [11]:
layer = keras.layers.Dense(3)
layer.trainable

True

time: 4.58 ms (started: 2021-08-26 21:48:00 +08:00)


在所有层和模型上，可以设置可训练属性（为 True 或 False）。 当设置为 False 时， layer.trainable_weights 属性为空：

In [13]:
layer = keras.layers.Dense(3)
layer.build(input_shape=(3, 3)) # Create the weights of the layer
layer.trainable

True

time: 24.9 ms (started: 2021-08-26 21:48:43 +08:00)


In [14]:
layer.trainable_weights

[<tf.Variable 'kernel:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.25013542,  0.4734645 , -0.29075813],
        [-0.4378314 , -0.23321152,  0.52709436],
        [-0.12084866,  0.53561974, -0.16229606]], dtype=float32)>,
 <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

time: 3.42 ms (started: 2021-08-26 21:48:53 +08:00)


In [15]:
layer.trainable = False

time: 658 µs (started: 2021-08-26 21:49:03 +08:00)


In [16]:
layer.trainable_weights

[]

time: 1.82 ms (started: 2021-08-26 21:49:11 +08:00)


在图层上设置可训练属性会递归地将其设置在所有子图层（self.layers 的内容）上。

1. 当用 fit() 训练时：

要使用 fit() 进行微调，您将：
* 实例化基本模型并加载预训练的权重
* 冻结基本模型
* 在顶部添加可训练层
* 调用 `compile()` 和 `fit()`

像这样：

In [ ]:
model = Sequential([
    ResNet50Base(input_shape=(32, 32, 3), weights='pretrained'),
    Dense(10),
])
model.layers[0].trainable = False  # Freeze ResNet50Base.

assert model.layers[0].trainable_weights == []  # ResNet50Base has no trainable weights.
assert len(model.trainable_weights) == 2  # Just the bias & kernel of the Dense layer.

model.compile(...)
model.fit(...)  # Train Dense while excluding ResNet50Base.

您可以使用 Functional API 或模型子类化 API 遵循类似的工作流程。 确保在更改 trainable 的值后调用 compile() 以便将您的更改考虑在内。 调用 compile() 将冻结模型训练步骤的状态。

2. 使用自定义训练循环时：

在编写训练循环时，请确保仅更新属于 `model.trainable_weights`（而不是所有 `model.weights`）的权重。

In [ ]:
model = Sequential([
    ResNet50Base(input_shape=(32, 32, 3), weights='pretrained'),
    Dense(10),
])
model.layers[0].trainable = False  # Freeze ResNet50Base.

# Iterate over the batches of a dataset.
for inputs, targets in dataset:
    # Open a GradientTape.
    with tf.GradientTape() as tape:
        # Forward pass.
        predictions = model(inputs)
        # Compute the loss value for this batch.
        loss_value = loss_fn(targets, predictions)

#     # Get gradients of loss wrt the *trainable* weights.
#     gradients = tape.gradient(loss_value, model.trainable_weights)
#     # Update the weights of the model.
#     optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    optimizer.minimize(loss_value, model.trainable_weights, tape=tape)

### `trainable` 和 `compile()` 之间的交互

在模型上调用 `compile()` 旨在“冻结”该模型的行为。 这意味着编译模型时的可训练属性值应在该模型的整个生命周期中保留，直到再次调用 `compile`。 因此，如果您更改可训练，请确保在您的模型上再次调用 `compile()` 以考虑您的更改。

例如，如果两个模型 A 和 B 共享一些层，并且：
* 模型 A 被编译
* 共享层上的`trainable`属性值已更改
* 模型B编译

然后模型 A 和 B 为共享层使用不同的可训练值。 这种机制对于大多数现有的 GAN 实现至关重要，它们可以：

In [ ]:
discriminator.compile(...)  # the weights of `discriminator` should be updated when `discriminator` is trained
discriminator.trainable = False
gan.compile(...)  # `discriminator` is a submodel of `gan`, which should not be updated

## `call()` 中的 `training` 参数和 `trainable` 属性之间有什么区别？
`training` 是 `call` 中的一个布尔参数，用于确定调用应在推理模式还是训练模式下运行。 例如，在训练模式下，`Dropout` 层应用随机 `dropout` 并重新调整输出。 在推理模式下，同一层什么都不做。 例子：

In [ ]:
y = Dropout(0.5)(x, training=True)  # Applies dropout at training time *and* inference time

`trainable` 是一个布尔层属性，它确定应更新层的可训练权重以最小化训练期间的损失。 如果 `layer.trainable` 设置为 `False`，则 `layer.trainable_weights` 将始终是一个空列表。 例子：

In [ ]:
model = Sequential([
    ResNet50Base(input_shape=(32, 32, 3), weights='pretrained'),
    Dense(10),
])
model.layers[0].trainable = False  # Freeze ResNet50Base.

assert model.layers[0].trainable_weights == []  # ResNet50Base has no trainable weights.
assert len(model.trainable_weights) == 2  # Just the bias & kernel of the Dense layer.

model.compile(...)
model.fit(...)  # Train Dense while excluding ResNet50Base.

如您所见，“推理模式与训练模式”和“层权重可训练性”是两个截然不同的概念。

你可以想象如下：一个 `dropout` 层，在训练期间通过反向传播学习缩放因子。 我们将其命名为 AutoScaleDropout。 该层将同时具有可训练状态，以及推理和训练中的不同行为。 由于可训练属性和训练调用参数是独立的，您可以执行以下操作：

In [ ]:
layer = AutoScaleDropout(0.5)

# Applies dropout at training time *and* inference time  
# *and* learns the scaling factor during training
y = layer(x, training=True)

assert len(layer.trainable_weights) == 1

In [ ]:
# Applies dropout at training time *and* inference time  
# with a *frozen* scaling factor

layer = AutoScaleDropout(0.5)
layer.trainable = False
y = layer(x, training=True)

### `BatchNormalization` 层的特例

考虑用于微调的模型冻结部分中的 `BatchNormalization` 层。

`BatchNormalization` 层的移动统计数据是否应该保持冻结或适应新数据一直存在争议。 从历史上看，`bn.trainable = False` 只会停止反向传播，但不会阻止训练时间统计更新。 经过大量测试，我们发现在微调用例中冻结移动统计数据通常更好。 从 TensorFlow 2.0 开始，设置 `bn.trainable = False` 也会强制该层在推理模式下运行。

此行为仅适用于 `BatchNormalization`。 对于其他每一层，权重可训练性和“推理与训练模式”保持独立。

## 在 `fit()` 中，如何计算验证拆分？
如果您将 `model.fit` 中的 `validation_split` 参数设置为例如 0.1，则使用的验证数据将是数据的最后 10%。 如果您将其设置为 0.25，它将是数据的最后 25%，等等。请注意，在提取验证拆分之前，数据并未打乱，因此验证实际上只是您输入中最后 x% 的样本 通过。

相同的验证集用于所有时期（在同一个 `fit` 调用中）。

请注意，仅当您的数据作为 Numpy 数组（不是 `tf.data.Datasets`，不可索引）传递时，`validation_split` 选项才可用。

## 在 `fit()` 中，数据在训练期间是否打乱？
如果您将数据作为 NumPy 数组传递，并且 `model.fit()` 中的 `shuffle` 参数设置为 `True`（这是默认值），则训练数据将在每个 `epoch` 全局随机打乱。

如果您将数据作为 `tf.data.Dataset` 对象传递，并且 `model.fit()` 中的 shuffle 参数设置为 True，则数据集将在本地进行混洗（缓冲混洗）。

使用 `tf.data.Dataset` 对象时，最好事先对数据进行混洗（例如，通过调用 `dataset = dataset.shuffle(buffer_size)）`以控制缓冲区大小。

验证数据永远不会被打乱。

## 使用 fit() 进行训练时，推荐的监控指标的方法是什么？
损失值和度量值通过调用 fit() 显示的默认进度条报告。 但是，在控制台中盯着更改 ascii 数字并不是最佳的指标监控体验。 我们建议使用 TensorBoard，它将显示漂亮的训练和验证指标图表，在训练期间定期更新，您可以从浏览器访问这些图表。

您可以通过 TensorBoard 回调将 TensorBoard 与 fit() 结合使用。

##  如果我需要自定义 `fit()` 的功能怎么办？
您有两个选择：

1. 编写一个低级自定义训练循环

如果您想控制每一个最后的小细节，这是一个不错的选择。 但它可能有点冗长。 例子：

In [ ]:
# 准备一个优化器。
optimizer = tf.keras.optimizers.Adam()
# Prepare a loss function.
loss_fn = tf.keras.losses.kl_divergence

# 迭代数据集的批次。
for inputs, targets in dataset:
    # 打开一个 GradientTape。
    with tf.GradientTape() as tape:
        # 前向传递
        predictions = model(inputs)
        # 计算该批次的损失值。
        loss_value = loss_fn(targets, predictions)

    # 获取权重的损失梯度。
    gradients = tape.gradient(loss_value, model.trainable_weights)
    # 更新模型的权重。
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

此示例不包括许多基本功能，例如显示进度条、调用回调、更新指标等。您必须自己执行此操作。 这一点都不难，但有点工作。

2. 子类模型类并覆盖`train_step`（和`test_step`）方法

如果您想使用自定义更新规则但仍想利用 `fit()` 提供的功能，例如回调、高效步骤融合等，这是一个更好的选择。

请注意，此模式不会阻止您使用 Functional API（甚至是 Sequential 模型）构建模型。

下面的示例显示了具有自定义 `train_step` 的功能模型。

In [20]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

class MyCustomModel(keras.Model):

    def train_step(self, data):
        # 解压数据。 它的结构取决于您的模型以及您传递给 `fit()` 的内容。
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # 计算损失值（损失函数在`compile()`中配置）
            loss = self.compiled_loss(y, y_pred,
                                      regularization_losses=self.losses)

        # 计算梯度
        trainable_vars = self.trainable_variables
#         gradients = tape.gradient(loss, trainable_vars)
        # 更新权重
#         self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.optimizer.minimize(loss, trainable_vars, tape = tape)
        # 更新指标（包括跟踪损失的指标）
        self.compiled_metrics.update_state(y, y_pred)
        # 将字典映射度量名称返回到当前值
        return {m.name: m.result() for m in self.metrics}

# 构造并编译 MyCustomModel 的一个实例
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = MyCustomModel(inputs, outputs)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# 像往常一样使用`fit`
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=10)

Epoch 1/10
32/32 [==============================] - 2s 12ms/step - loss: 0.4085 - accuracy: 0.0000e+00
Epoch 2/10
32/32 [==============================] - 0s 11ms/step - loss: 0.2021 - accuracy: 0.0000e+00
Epoch 3/10
32/32 [==============================] - 0s 11ms/step - loss: 0.1915 - accuracy: 0.0000e+00
Epoch 4/10
32/32 [==============================] - 0s 11ms/step - loss: 0.1871 - accuracy: 0.0000e+00
Epoch 5/10
32/32 [==============================] - 0s 11ms/step - loss: 0.1827 - accuracy: 0.0000e+00
Epoch 6/10
32/32 [==============================] - 0s 11ms/step - loss: 0.1783 - accuracy: 0.0000e+00
Epoch 7/10
32/32 [==============================] - 0s 11ms/step - loss: 0.1738 - accuracy: 0.0000e+00
Epoch 8/10
32/32 [==============================] - 0s 11ms/step - loss: 0.1692 - accuracy: 0.0000e+00
Epoch 9/10
32/32 [==============================] - 0s 11ms/step - loss: 0.1647 - accuracy: 0.0000e+00
Epoch 10/10
32/32 [==============================] - 0s 10ms/step - loss:

time: 6.18 s (started: 2021-08-26 22:27:57 +08:00)


您还可以轻松添加对样本加权的支持：

In [ ]:
class MyCustomModel(keras.Model):

    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        if len(data) == 3:
            x, y, sample_weight = data
        else:
            x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value.
            # The loss function is configured in `compile()`.
            loss = self.compiled_loss(y, y_pred,
                                      sample_weight=sample_weight,
                                      regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        self.compiled_metrics.update_state(
            y, y_pred, sample_weight=sample_weight)

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}
    
# Construct and compile an instance of MyCustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = MyCustomModel(inputs, outputs)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# You can now use sample_weight argument
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
sw = np.random.random((1000, 1))
model.fit(x, y, sample_weight=sw, epochs=10)

同样，您也可以通过覆盖 test_step 来自定义评估：

In [ ]:
class MyCustomModel(keras.Model):

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(
          y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}

## 如何以混合精度训练模型？
Keras 内置支持在 GPU 和 TPU 上进行混合精度训练。 请参阅此详尽指南。

# 建模相关问题
## 如何获得中间层的输出（特征提取）？
在 Functional API 和 Sequential API 中，如果一个层被调用了一次，你可以通过 `layer.output` 检索它的输出，通过 `layer.input` 检索它的输入。 这使您能够快速实例化特征提取模型，如下所示：

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.models.Sequential([
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(2),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(32, 3, activation='relu'),
    layers.GlobalMaxPooling2D(),
    layers.Dense(10),
])
extractor = keras.Model(inputs=model.inputs,
                        outputs=[layer.output for layer in model.layers])
features = extractor(data)

自然，这对于作为覆盖调用的 Model 子类的模型是不可能的。

这是另一个示例：实例化一个返回特定命名层输出的模型：

In [ ]:
model = ...  # create the original model

layer_name = 'my_layer'
intermediate_layer_model = keras.Model(inputs=model.input,
                                       outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model(data)

## 如何在 Keras 中使用预训练模型？
您可以利用 keras.applications 中可用的模型，或 TensorFlow Hub 上可用的模型。 TensorFlow Hub 与 Keras 集成良好。

## 如何使用有状态的 RNN？
使 RNN 有状态意味着每批样本的状态将被重用作为下一批样本的初始状态。

因此，在使用有状态 RNN 时，假设：
* 所有批次的样品数量相同
* 如果 x1 和 x2 是连续批次的样本，则 x2[i] 是 x1[i] 的后续序列，对于每个 i。

要在 RNN 中使用状态性，您需要：
* 通过将 `batch_size` 参数传递给模型中的第一层，明确指定您正在使用的批量大小。 例如。 `batch_size=32` 用于 10 个时间步长的 32 个样本批次序列，每个时间步长有 16 个特征。
* 在您的 RNN 层中设置 `stateful=True`。
* 调用 `fit()` 时指定 `shuffle=False`。

要重置累积的状态：
* 使用 `model.reset_states()` 重置模型中所有层的状态
* 使用 `layer.reset_states()` 重置特定有状态 RNN 层的状态

例子：

In [27]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

x = np.random.random((32, 21, 16)) # 这是我们的输入数据，形状为 (32, 21, 16)
# 我们将把它以长度为 10 的序列提供给我们的模型

model = keras.Sequential()
model.add(layers.LSTM(32, input_shape=(10, 16), batch_size=32, stateful=True))
model.add(layers.Dense(16, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# 我们训练网络在给定前 10 个时间步长的情况下预测第 11 个时间步长：
model.train_on_batch(x[:, :10, :], np.reshape(x[:, 10, :], (32, 16)))

# 网络状态发生了变化。 我们可以提供后续序列：
model.train_on_batch(x[:, 10:20, :], np.reshape(x[:, 20, :], (32, 16)))

# 让我们重置 LSTM 层的状态：
model.reset_states()

# 在这种情况下的另一种方法：
model.layers[0].reset_states()

time: 3 s (started: 2021-08-26 22:42:39 +08:00)


请注意，方法 `predict、fit、train_on_batch` 等都会更新模型中状态层的状态。 这使您不仅可以进行有状态训练，还可以进行有状态预测。